<a href="https://colab.research.google.com/github/Austin-Deccentric/Team_Swift/blob/master/Article_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -U turicreate

     |████████████████████████████████| 90.5MB 1.2MB/s 
     |████████████████████████████████| 2.1MB 40.3MB/s 
     |████████████████████████████████| 3.4MB 32.2MB/s 
     |████████████████████████████████| 17.3MB 41.2MB/s 
     |████████████████████████████████| 23.8MB 1.2MB/s 
     |████████████████████████████████| 327kB 37.7MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320847 sha256=65e74fcde4f0c1659cdc1a04510ce7d277886c9e082d8b330344a16ef481ec41
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mxnet 1.1.0.post0 has req

In [0]:
import pandas as pd
import numpy as np
import turicreate as tc

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/hngi6/Recommendation_system/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/hngi6/Recommendation_system


In [0]:
post = pd.read_csv('posts.csv')
ratings = pd.read_csv('ratings.csv')

In [7]:
post.head(10)

,user_id,title,content,tags,slug
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,NaN,what-i-have-learnt-so-far-on-html-985
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,NaN,my-laziness-in-the-open-029
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,NaN,my-task-2-649
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",NaN,task-2-7e5
5,329,My Journey on HTML,Using the Hyper Text Markup Language (HTML) ha...,NaN,my-journey-on-html-ce5
6,2040,StartNG HTML Exposition,![](/storage/2040/images/img-kf6sy3kvg0.png)![...,NaN,startng-html-exposition-4d5
7,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,html,my-task-2-649
8,2040,StartNG HTML task,<p> </p>\n<p>I have learned a lot about HTML a...,NaN,startng-html-task-8e2
9,2040,On StartNG Pre-Internship,**What I have learned so far**\n\nI have learn...,"StartNG pre-internship, Developer, Engineering...",on-startng-pre-internship-645


In [0]:
col = ['tags', 'slug']

post = post.drop(post[col], axis=1)

In [9]:
post.head()

,user_id,title,content
0,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...
1,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ..."
2,1310,My Laziness In The Open,I have not been attending classes on the HNG c...
3,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...
4,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str..."


In [10]:
ratings.head()

,user_id,post_id,Ratings
0,1,995,5
1,1,355,3
2,1,996,4
3,1,996,5
4,1,996,4


In [0]:
#n_users = merge_col.user_id.unique().shape[0]
#n_posts = merge_col.post_id.unique().shape[0]
#print(n_users)
#print(n_posts)

In [12]:
merge_col = pd.merge(post, ratings, on='user_id')


merge_col.head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143,3
3,4,Good Afternoon People,Hey,999,4
4,4,Good Afternoon People,Hey,278,3


In [13]:
#number of unique users
n_users = post['user_id'].unique() ; print("number of unique users" , len(n_users))

number of unique users 308


In [14]:
#number of unquie posts
n_posts = post['title'].unique() ; print("Number of unique posts" , len(n_posts))

Number of unique posts 658


In [0]:
merge_col.to_csv('merged_data.csv')

In [0]:
merged_data = pd.read_csv('merged_data.csv')

In [20]:
merged_data.head()

,user_id,title,content,post_id,Ratings
0,2191,What you have learned so far,"1. I have learnt how to publish and article, ...",143,3
1,2191,What you wish to still learn,1. I want more insight on clodinary and so on,143,3
2,2191,What you wish should be improved,1. For now am OK with the setting,143,3
3,4,Good Afternoon People,Hey,999,4
4,4,Good Afternoon People,Hey,278,3


In [21]:
data = tc.SFrame.read_csv('merged_data.csv')

Finished parsing file /gdrive/My Drive/hngi6/Recommendation_system/merged_data.csv

Parsing completed. Parsed 100 lines in 0.04922 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /gdrive/My Drive/hngi6/Recommendation_system/merged_data.csv

Parsing completed. Parsed 622 lines in 0.028372 secs.

**Building The Model**

In [0]:
#using the ranking factorization 
train_data, validation_data = tc.recommender.util.random_split_by_user(data, 'user_id', 'title')

In [29]:
ranking_model = tc.ranking_factorization_recommender.create(train_data, user_id ='user_id', item_id ='title',target = 'Ratings')

Preparing data set.

Data has 505 observations with 8 users and 45 items.

Data prepared in: 0.01666s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.17343

Final training RMSE: 0.656017

In [28]:
#Building item similarity using Cosine similarity
item_sim_model = tc.item_similarity_recommender.create(train_data, user_id='user_id', item_id='content', target='Ratings', similarity_type='cosine')

Warning: Ignoring columns title, post_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 505 observations with 8 users and 51 items.

Data prepared in: 0.003793s

...

| 712us                               | 100              | 51              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.00128s

**Comparing The two trained models**

In [30]:
tc.recommender.util.compare_models(validation_data, [ranking_model,item_sim_model], metric='rmse')

PROGRESS: Evaluate model M0

Overall RMSE: 0.6714510153060398

Per User RMSE (best)
+---------+---------------------+-------+
| user_id |         rmse        | count |
+---------+---------------------+-------+
|    3    | 0.26595761735749474 |   43  |
+---------+---------------------+-------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---------+--------------------+-------+
|    7    | 1.2595800365271255 |   7   |
+---------+--------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (best)
+---------------+----------------------+-------+
|     title     |         rmse         | count |
+---------------+----------------------+-------+
| Lucid is live | 0.013970796689121379 |   1   |
+---------------+----------------------+-------+
[1 rows x 3 columns]


Per Item RMSE (worst)
+-------------------------------+--------------------+-------+
|             title             |        rmse        | coun

[{'rmse_by_item': Columns:
  	title	str
  	rmse	float
  	count	int
  
  Rows: 29
  
  Data:
  +------------------------------+----------------------+-------+
  |            title             |         rmse         | count |
  +------------------------------+----------------------+-------+
  |        Lucid is live         | 0.013970796689121379 |   1   |
  |    My Experience In HTML     |  0.4468364043048592  |   9   |
  |       SHALLOW THINKING       |  0.8575637876327495  |   1   |
  | My Internship Journey So Far | 0.30400101211977787  |   5   |
  |      Took a long while       |  0.3383643758278332  |   1   |
  |     this is a test post      |  0.8376913224283208  |   7   |
  |        Lucid is back         | 0.021811943039410208 |   3   |
  |       Welcome to Lucid       |  0.6389040360330667  |   7   |
  |             Test             |  0.2121500161745069  |   1   |
  |    New features incoming     | 0.09328770628219087  |   4   |
  +------------------------------+----------------

**Now to Test the System**

In [32]:
all_users_rank = ranking_model.recommend(k=5)
all_users_rank

user_id,title,score,rank
2191,First,3.86406084990295,1
2191,third,3.8598201452025993,2
2191,Took a long while,3.8518344117531402,3
2191,second,3.7884046901220305,4
2191,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.67769249810072,5
4,First,3.963874437866531,1
4,third,3.851103971732579,2
4,Took a long while,3.826869505477868,3
4,second,3.784346805905722,4
4,html test,3.650228773264013,5


Making Recommendations for new users

In [34]:
#use-id 2020 is a new user
rank_factor = rating_model.recommend(users=[2020], k=3)
rank_factor

user_id,title,score,rank
2020,YOUR HIGH IQ WILL KILLYOUR STARTUP ...,4.044153985587679,1
2020,A CHINESE VILLAGER WHOSELLS MORE SOFTWARE D ...,3.98819037994536,2
2020,BUILD AROUND YOURDISCOVERY MECHANISM ...,3.8547673719206674,3
